파이프라인을 사용하면 복잡한 모델링 코드를 간단한 한 단계로 만들 수 있다.  
파이프라인을 쓰지않는 사람들이 많지만 쓸때의 이점이 있다.  
1. 각 단계를 수동으로 처리할 필요가 없다.
2. 과정을 빠뜨려 오류가 발생한 경우가 적다.
3. 프로토타입에서 배치가능한 모델로 만드는 것은 꽤 어려운 일이 될 수 있다.
4. 모델 검증을 위한 추가 옵션

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("../datafile/melb_data.csv")

y = data.Price
X = data.drop(["Price"], axis=1)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ["int64","float64"]]

my_cols = categorical_cols + numerical_cols

X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

X_train.head()


Type Method             Regionname  Rooms  Distance  Postcode  Bedroom2  \
12167    u      S  Southern Metropolitan      1       5.0    3182.0       1.0   
6524     h     SA   Western Metropolitan      2       8.0    3016.0       2.0   
8413     h      S   Western Metropolitan      3      12.6    3020.0       3.0   
2919     u     SP  Northern Metropolitan      3      13.0    3046.0       3.0   
6043     h      S   Western Metropolitan      3      13.3    3020.0       3.0   

       Bathroom  Car  Landsize  BuildingArea  YearBuilt  Lattitude  \
12167       1.0  1.0       0.0           NaN     1940.0  -37.85984   
6524        2.0  1.0     193.0           NaN        NaN  -37.85800   
8413        1.0  1.0     555.0           NaN        NaN  -37.79880   
2919        1.0  1.0     265.0           NaN     1995.0  -37.70830   
6043        1.0  2.0     673.0         673.0     1970.0  -37.76230   

       Longtitude  Propertycount  
12167    144.9867        13240.0  
6524     144.9005         6380.0  
8413     144.8220         3755.0  
2919     144.9158         8870.0  
6043     144.8272         4217.0

### **1.Define Preprocessing Steps**

In [19]:
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numerical_transformer = SimpleImputer(strategy="constant")

categorical_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent")),("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(transformers=[("num", numerical_transformer, numerical_cols),("cat",categorical_transformer, categorical_cols)])


* simpleimputer : strategy의 default는 mean / constant는 누락된 값을 fill_value로 바꿈

### **2.Define the Model**

In [20]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### **3.Create and Evaluate the Pipeline**

In [21]:
from sklearn.metrics import mean_absolute_error

my_pipeline = Pipeline(steps=[("preprocessor",preprocessor),("model",model)])

my_pipeline.fit(X_train, y_train)

preds = my_pipeline.predict(X_valid)

score = mean_absolute_error(y_valid, preds)

print(score)

160679.18917034855
